In [1]:
%run optimizers/Optimizer.ipynb

class ShapeEstimation(Optimizer):
    
    def __init__(self, mean, pcaBasis, pcaVariance):
        Optimizer.__init__(self, mean, pcaBasis, pcaVariance)
    
    def open(self):
        self.session = tf.Session()
        self._setVariables()
    
    def close(self):
        self.session.close()
    
    def optimize(self, targetFeatures):
        self._setTargetFeaturesComponent(targetFeatures)
        self._setCComponent()
        self._setHComponent(targetFeatures)
        self.k = tf.linalg.lstsq(
                matrix=self.C,
                rhs=self.h)
        return (np.array(self.session.run(self.k)),
                self.r,
                self.t,
                self.s)
    
    def update(self, a, r, t, s):
        self.r = r
        self.t = t
        self.s = s
        
    def _setVariables(self):
        self.C = tf.Variable(np.zeros((2*self.numFeatures, self.numComponents)), name='C', trainable=False)
        self.h = tf.Variable(np.zeros((2*self.numFeatures, 1)), name='h', trainable=False)
        self.targetFeatures = tf.Variable(np.zeros((2*self.numFeatures, 1), dtype=np.float64), name='targetFeatures', trainable=False)
        ai, ri, ti, si = Optimizer.getDefaults(self.numComponents)
        self.r = ri
        self.t = ti
        self.s = si
    
    def _setTargetFeaturesComponent(self, targetFeatures):
        self.session.run(self.targetFeatures.assign(np.reshape(targetFeatures, (2*self.numFeatures, 1))))
    
    def _setCComponent(self):
        C = np.zeros((2*self.numFeatures, self.numComponents))
        for i in range(self.numFeatures):
            C[2*i] = (self.r[0,0] * self.pcaBasis[3*i,:]
                    + self.r[0,1] * self.pcaBasis[3*i+1,:]
                    + self.r[0,2] * self.pcaBasis[3*i+2,:])
            C[2*i+1] = (self.r[1,0] * self.pcaBasis[3*i,:]
                    + self.r[1,1] * self.pcaBasis[3*i+1,:]
                    + self.r[1,2] * self.pcaBasis[3*i+2,:])
        C *= self.s
        self.session.run(self.C.assign(C))
        
    def _setHComponent(self, targetFeatures):
        h = np.zeros((2*self.numFeatures, 1))
        for i in range(self.numFeatures):
            h[2*i]   = targetFeatures[2*i] - self.s * (np.matmul(self.r[0], self.mean[3*i:3*i+3]) + self.t[0,0])
            h[2*i+1] = targetFeatures[2*i+1] - self.s * (np.matmul(self.r[1], self.mean[3*i:3*i+3]) + self.t[1,0])
        self.session.run(self.h.assign(h))
    

ERROR:root:File `'optimizers/Optimizer.ipynb.py'` not found.


NameError: name 'Optimizer' is not defined